In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch

from bo_functions import best_error_advanced
from bo_functions import calc_ei_advanced
from bo_functions import ExactGPModel
from bo_functions import train_GP_model
from bo_functions import calc_GP_outputs
from bo_functions import calc_y_expected

from bo_plotters import plotter_adv
from bo_plotters import basic_plotter
from bo_plotters import ei_plotter_basic
from bo_plotters import y_plotter_adv
from bo_plotters import stdev_plotter_adv
from bo_plotters import error_plotter_adv
from bo_plotters import ei_plotter_adv

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.tri as mtri
import matplotlib

In [2]:
#Pull x and Y data from CSV
#Pull x data from CSV
exp_data_doc = "exp_data.csv"
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1]
Yexp = exp_data[:,2]

n = len(Xexp)
print(n)
Theta_True = np.array([1,-1])

5


In [3]:
#Create training and test data
train_data_doc = "train_3_in_data.csv"
train_data = np.array(pd.read_csv(train_data_doc, header=0,sep=","))
# print(train_data)
train_theta = train_data[:,1:3]
train_p = torch.tensor(train_data[:,1:4])
train_y = torch.tensor(train_data[:,4])
print(train_p)
print(train_y)

tensor([[ 1.8567e+00,  7.6583e-01,  1.2417e+00],
        [-1.5359e+00, -1.4589e+00,  4.0733e-01],
        [ 1.1929e+00, -1.2549e+00, -1.8451e+00],
        [-1.9292e+00,  8.9024e-01,  1.6229e+00],
        [ 2.7640e-01,  6.0036e-02,  1.7340e+00],
        [ 6.7009e-01, -6.7689e-01,  1.1550e+00],
        [-2.6864e-01,  6.1656e-01, -1.7169e+00],
        [ 8.2577e-01,  1.3867e+00, -3.5378e-01],
        [ 1.0064e+00,  3.3627e-01,  6.2007e-01],
        [-1.6244e-03,  1.2003e-01,  7.8550e-01],
        [-1.3450e+00,  4.2636e-01, -8.2567e-01],
        [-9.8063e-01, -1.3159e-01, -4.0761e-01],
        [-1.0562e+00, -1.9284e+00,  3.4419e-01],
        [-1.7425e+00,  1.0433e+00, -1.5221e-01],
        [-1.3687e+00,  1.9121e+00,  2.0146e-01],
        [-1.6021e-01, -7.7287e-01, -6.7441e-01],
        [ 1.2909e+00,  1.7409e+00, -1.3370e+00],
        [-7.1961e-01, -9.3770e-01,  1.4750e+00],
        [-5.2986e-01,  1.5590e+00,  9.2303e-01]], dtype=torch.float64)
tensor([  5.4011,  -0.8001, -12.7549,   3.4883,

In [4]:
# initialize likelihood and model
##Assumes a homoskedastic noise model p(y | f) = f + noise
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# We will use the simplest form of GP model, exact inference
#Defines our model in terms of the class parameters in bo_functions
model = ExactGPModel(train_p, train_y, likelihood)

In [5]:
#Set number of training iterations and train GP
iterations = 500
train_GP_model(model,likelihood, train_p, train_y, iterations)

In [6]:
# Get into evaluation (predictive posterior) mode
#Puts model in evaluation mode
model.eval()
#Puts likelihood in evaluation mode
likelihood.eval();

In [7]:
#Create Meshgrid
p = 10 #Any bigger than 10 and the kernel just dies
Theta1 = np.linspace(-2,2,p)
Theta2 = np.linspace(-2,2,p)

theta_mesh = np.array(np.meshgrid(Theta1,Theta2))
theta_space = torch.tensor(theta_mesh.T.reshape(-1,2))

In [8]:
theta1_mesh = theta_mesh[0]
theta2_mesh = theta_mesh[1]

# for i in range(p):
#     for j in range(p):
#         print([theta1_mesh[i,j],theta2_mesh[i,j]])

In [9]:
#Define f_bar and f(x)
f_bar = Yexp #(1xn)
#Will compare the rigorous solution and approximation later (multidimensional integral over each experiment using a sparse grid)

#Create an array in which to store expected improvement values
EI = np.zeros((p,p)) #(p1 x p2)
# Loop over theta 1
for i in range(p):
    #Loop over theta2
    for j in range(p):
        #Create array to store error values
        error = np.zeros(n)
        #Loop over Xexp
        for k in range(n):
            #Evaluate GP at a point p = [Theta1,Theta2,Xexp]
            eval_point = []
            eval_point.append([theta1_mesh[i,j],theta2_mesh[i,j],Xexp[k]])
            eval_point = np.array(eval_point)
            GP_Outputs = calc_GP_outputs(model, likelihood, eval_point[0:1])
            model_mean = GP_Outputs[3].numpy()[0] #1xn
            model_variance= GP_Outputs[1].numpy()[0] #1xn
#             print(eval_point,model_mean,model_variance)
            #Compute error for that point
#             print(eval_point,model_mean, f_bar[k])
            error[k] = -(f_bar[k] - model_mean)**2
#             print(error[k])
        #Define best_error as the maximum value in the error array
        best_error = -max(error)
#         print(best_error)
        #Loop over Xexp
        for k in range(n):
            #Caclulate EI for each value n given the best error
#             print(i,j,k)
            eval_point = []
            eval_point.append([theta1_mesh[i,j],theta2_mesh[i,j],Xexp[k]])
            eval_point = np.array(eval_point)
#             print(eval_point)
            GP_Outputs = calc_GP_outputs(model, likelihood, eval_point[0:1])
            model_mean = GP_Outputs[3].numpy()[0] #1xn
            model_variance= GP_Outputs[1].numpy()[0] #1xn
#             print(eval_point,model_mean,model_variance,Yexp[k])
            EI[i,j] += calc_ei_advanced(best_error, model_mean, model_variance, Yexp[k])
#             print(EI[i,j])


print(np.min(EI))
argmin = np.array(np.where(EI == np.amin(EI)))
Theta_1_Opt = theta1_mesh[argmin[0],argmin[1]]
Theta_2_Opt = theta2_mesh[argmin[0],argmin[1]]
Theta_GP_Opt = np.array([Theta_1_Opt,Theta_2_Opt])
print("The GP estimates \nTheta1, Theta2 = \n")
print(Theta_GP_Opt)

argmax = np.array(np.where(EI == np.amax(EI)))
Theta_1_Best = theta1_mesh[argmax[0],argmax[1]]
Theta_2_Best = theta2_mesh[argmax[0],argmax[1]]
Theta_Best = np.array([Theta_1_Best,Theta_2_Best])
print("The GP estimates the highest EI is at \nTheta1, Theta2 = \n ")
print(Theta_Best)

2.816586573942472e-07
The GP estimates 
Theta1, Theta2 = 

[[0.66666667]
 [0.22222222]]
The GP estimates the highest EI is at 
Theta1, Theta2 = 
 
[[ 2.]
 [-2.]]


In [10]:
ei_plotter_adv(theta_mesh, EI, Theta_True, Theta_Best,train_theta,plot_train=True)

TypeError: ei_plotter_adv() missing 1 required positional argument: 'train_p'

## Analysis of Expected Improvement
 - Expected Improvement is largest farther from the edges
  - This is rational because you can't explore any further than the edges
 - Expected Improvement increases as error decreases
  - This is rational because as error decreases, more exploitation is possible
 - This means we are most likely to sample in the middle, farthet from the edges


## Analysis of Standard Deviation
 - The GP estimates that the standard deviation is lowest at points that were directly tested
  - This can be rationalized by the way that the contour plot is drawn
 - Standard deviation is smallest away from the edges and larger towards them
  - This is rationalized by the fact that there are less neighbors that the GP is tested and trained with at the boundaries
 - The more points that get tested, the more the standard deviations will decrease

## Analysis of Error Magnitude
 - The GP emulator is most inaccurate when all values of $\bar{p}$ are at their maximum. 
  - In general, the GP is less accurate at extreme points, this is rationalized by the fact that there are less neighbors that the GP is tested and trained with at the boundaries
 - The GP emulator is most accurate when x is at it's maximum, but $\bar{\Theta}= 0$
  - This is rationalized by the fact that multiple terms become zero if any of the values of $\bar{p}$ are zero 
 - GP error is mostly very high, as more iterations are added, these will decrease

## Analysis of GP Emulator (Model y)
 - The GP emulator correctly captures that y increases as $\bar{p}$ increases. This tells us that this GP emulator model could be viable
  - The GP emulator correctly estimates where the lowest y is achieved, but not the actual value of y
  - The GP emulator slightly mistakes where the most positive value of y is, and does not predict the actual value of y
 - The model as it is is inaccurate, BO should increase the accuracy of the emulator